In [1]:
import os
from datetime import datetime
import shutil

In [2]:
from configs.config_object import ConfigObject
from grid_and_bathy import get_dz_grid, get_grid
from surface_forcings import extract_and_save_surface_forcings, download_weather_reanalysis, download_weather_forecast
from initial_conditions import download_profile_idronaut_datalakes, create_initial_temperature_from_measure_profile
from run_preprocessing import write_data_config_files, copy_template, remove_all_files_and_folders, write_size_config_files
from utils import modify_arguments, convert_binary_files

## Load config

In [3]:
config = ConfigObject('configs/config.json')
output_folder = './99-output/'
template_folder = os.path.join('./00-template_mitgcm/', config.template_folder)

In [4]:
parsed_start_date = datetime.strptime(config.start_date, '%Y%m%d')
parsed_end_date = datetime.strptime(config.end_date, '%Y%m%d')
parsed_ref_date = datetime.strptime(config.reference_date, '%Y%m%d')

sim_duration_in_second = (parsed_end_date - parsed_start_date).total_seconds()
start_time_in_second_from_ref_date = (parsed_start_date - parsed_ref_date).total_seconds()
end_time_in_second_from_ref_date = (parsed_end_date - parsed_ref_date).total_seconds()

## Initialize output folder

In [25]:
remove_all_files_and_folders(output_folder)
copy_template('./00-template_mitgcm/geneva_base', output_folder)
copy_template(template_folder, output_folder)
config.write_metadata_to_file(os.path.join(output_folder, 'metadata_simulation.txt'))

## Get MITgcm grid

#In case the grid has not been created yet
build_and_save_mitgcm_grid(config.paths.grid_folder_path, 
                           config.Nx, 
                           config.Ny, 
                           config.grid_resolution, 
                           config.x0_ch1903, 
                           config.y0_ch1903, 
                           config.x1_ch1903, 
                           config.y1_ch1903)

In [6]:
mitgcm_grid = get_grid(config.paths.grid_folder_path)

## Build binary files

In [7]:
binary_data_folder = os.path.join(output_folder, 'binary_data')
if not os.path.exists(binary_data_folder):
    os.makedirs(binary_data_folder)
    print(f"Directory '{binary_data_folder}' created.")

Directory './99-output/binary_data' created.


**Get bathymetry**

In [8]:
#To create bathymetry, see notebook 'create_bathy'
shutil.copy(config.paths.bathy_path, os.path.join(binary_data_folder, 'bathy.bin'))

'./99-output/binary_data\\bathy.bin'

**Get surface forcings**

In [9]:
buffer = config.weather_download_buffer
if config.weather_model_type == 'reanalysis':
    download_weather_reanalysis(config.weather_api_base_url, 
                                config.start_date, config.end_date, 
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather reanalysis')
elif config.weather_model_type == 'forecast':
    download_weather_forecast(config.weather_api_base_url, 
                                config.start_date,  
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather forecasts')

Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_U.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_T_2M.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_GLOB.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_V.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_RELHUM_2M.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240402_20240402_T_2M.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_20240401_PMSL.json
Data already exists: C:\Users\leroquan\Documents\Data\meteo\COSMO reanalysis\raw_from_alplakes_api\20240401_2024

In [10]:
time_format = 'UTC'
extract_and_save_surface_forcings(binary_data_folder, 
                                  config.start_date, 
                                  config.end_date, 
                                  config.paths.raw_results_from_api_folder, 
                                  mitgcm_grid,
                                  1, # Number of cores used in interpolation
                                  config.weather_model_type)

Interpolating U to grid...
Saving u10...
Interpolating V to grid...
Saving v10...
Interpolating GLOB to grid...
Saving swdown...
Interpolating T_2M to grid...
Saving atemp...
Interpolating PS to grid...
Saving apressure...
Computing specific humidity (aqh)...
Saving aqh...
Computing longwave radiation (lwdown)...
Saving lwdown...
Done computing binary data.


**Convert to little endian if necessary** (for cscs computation)

In [11]:
# TO DO : implement check to see whether the datatypes are correct
if config.endian_type == 'little_endian':
    convert_binary_files(os.path.join(output_folder, 'binary_data'), '>f8', '<f8')

### Get initial conditions

In [26]:
file_date, measured_temperature = download_profile_idronaut_datalakes(parsed_start_date)
print('Initial conditions from Idronaut file from ', file_date)
dz_grid = get_dz_grid(config.paths.dz_grid_csv_path)
shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, measured_temperature)

Initial conditions from Idronaut file from  2024-03-31 22:57:11.900000


### Get pickup files

In [27]:
if config.with_pickup:
    dt_sim_in_second = 4
    pickup_number = f"pickup.{str(int(start_time_in_second_from_ref_date / dt_sim_in_second)).zfill(10)}"
else:
    pickup_number = ""
print(pickup_number)

### Adapt config files

In [28]:
modify_arguments('!reference_date!', [config.reference_date], os.path.join(output_folder, 'run_config/data.cal'))

In [29]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run_config/data.exf'))

In [30]:
#TO DO: write output type (binary or netcdf -->   diag_mnc = .FALSE./.TRUE. in data.diagnostics)

In [31]:
write_data_config_files(os.path.join(output_folder, 'run_config/data'), 
                        shaped_temp_initial, 
                        start_time_in_second_from_ref_date, 
                        end_time_in_second_from_ref_date,
                        pickup_number,
                        dz_grid, 
                        config.grid_resolution)

In [32]:
write_size_config_files(os.path.join(output_folder, 'code/SIZE.h'), 
                        config.Px, 
                        config.Py, 
                        config.Nx, 
                        config.Ny, 
                        len(dz_grid.flatten()))

## Next steps

- Copy-paste the folders in "99-output" to the cscs server 
- Change #SBATCH --time=00:05:00 in run/submit-daint-short.sh
- Open CSCS putty console:

export MITGCM_ROOTDIR=/scratch/snx3000/aleroqua/MITgcm-checkpoint67z/

cd /scratch/snx3000/aleroqua/xx/build
$MITGCM_ROOTDIR/tools/genmake2 -mods ../code -mpi -enable=mnc -of ${MITGCM_ROOTDIR}/tools/build_options/linux_ia64_cray_archer
make depend
make -j 8

cd ..
rm -r run
mkdir run
cd run
ln -s ../pickup/* .
ln -s ../run_config/* .
cp ../build/mitgcmuv .

sbatch submit-daint-short.sh

### TO DO

In [ ]:
# add selection of pickup file
# add selection of pickup frequency, diagnostic frequency
# add selection of output type (mnc = true or false)
# add automatic change of computing time? (submit-daint)